In [ ]:
# !pip install kfp==1.8.22

In [1]:
import json
import re
import requests
import kfp
import os
import kubernetes as k8s

from urllib.parse import urlsplit
from kfp.onprem import use_k8s_secret
from kfp import components
from kfp.components import load_component_from_file
from kfp import dsl
from kfp import compiler


def get_istio_auth_session(url: str, username: str, password: str) -> dict:
    """
    Determine if the specified URL is secured by Dex and try to obtain a session cookie.
    WARNING: only Dex `staticPasswords` and `LDAP` authentication are currently supported
             (we default default to using `staticPasswords` if both are enabled)

    :param url: Kubeflow server URL, including protocol
    :param username: Dex `staticPasswords` or `LDAP` username
    :param password: Dex `staticPasswords` or `LDAP` password
    :return: auth session information
    """
    # define the default return object
    auth_session = {
        "endpoint_url": url,  # KF endpoint URL
        "redirect_url": None,  # KF redirect URL, if applicable
        "dex_login_url": None,  # Dex login URL (for POST of credentials)
        "is_secured": None,  # True if KF endpoint is secured
        # Resulting session cookies in the form "key1=value1; key2=value2"
        "session_cookie": None,
    }

    # use a persistent session (for cookies)
    with requests.Session() as s:
        ################
        # Determine if Endpoint is Secured
        ################
        resp = s.get(url, allow_redirects=True)
        if resp.status_code != 200:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for GET against: {url}"
            )

        auth_session["redirect_url"] = resp.url

        # if we were NOT redirected, then the endpoint is UNSECURED
        if len(resp.history) == 0:
            auth_session["is_secured"] = False
            return auth_session
        else:
            auth_session["is_secured"] = True

        ################
        # Get Dex Login URL
        ################
        redirect_url_obj = urlsplit(auth_session["redirect_url"])

        # if we are at `/auth?=xxxx` path, we need to select an auth type
        if re.search(r"/auth$", redirect_url_obj.path):
            #######
            # TIP: choose the default auth type by including ONE of the following
            #######

            # OPTION 1: set "staticPasswords" as default auth type
            redirect_url_obj = redirect_url_obj._replace(
                path=re.sub(r"/auth$", "/auth/local", redirect_url_obj.path)
            )
            # OPTION 2: set "ldap" as default auth type
            # redirect_url_obj = redirect_url_obj._replace(
            #     path=re.sub(r"/auth$", "/auth/ldap", redirect_url_obj.path)
            # )

        # if we are at `/auth/xxxx/login` path, then no further action is needed (we can use it for login POST)
        if re.search(r"/auth/.*/login$", redirect_url_obj.path):
            auth_session["dex_login_url"] = redirect_url_obj.geturl()

        # else, we need to be redirected to the actual login page
        else:
            # this GET should redirect us to the `/auth/xxxx/login` path
            resp = s.get(redirect_url_obj.geturl(), allow_redirects=True)
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for GET against: {redirect_url_obj.geturl()}"
                )

            # set the login url
            auth_session["dex_login_url"] = resp.url

        ################
        # Attempt Dex Login
        ################
        resp = s.post(
            auth_session["dex_login_url"],
            data={"login": username, "password": password},
            allow_redirects=True,
        )
        if len(resp.history) == 0:
            raise RuntimeError(
                f"Login credentials were probably invalid - "
                f"No redirect after POST to: {auth_session['dex_login_url']}"
            )

        # store the session cookies in a "key1=value1; key2=value2" string
        auth_session["session_cookie"] = "; ".join(
            [f"{c.name}={c.value}" for c in s.cookies]
        )

    return auth_session


KUBEFLOW_ENDPOINT = os.environ.get(
    "KUBEFLOW_ENDPOINT", "https://kubeflow.ui.kubeflow.awsuser.cloudns.ph/"
)
KUBEFLOW_USERNAME = os.environ.get("KUBEFLOW_USERNAME", "user@example.com")
KUBEFLOW_PASSWORD = os.environ.get("KUBEFLOW_PASSWORD", "12341234")

print(f"fetching auth session from {KUBEFLOW_ENDPOINT}")

auth_session = get_istio_auth_session(
    url=KUBEFLOW_ENDPOINT, username=KUBEFLOW_USERNAME, password=KUBEFLOW_PASSWORD
)

INGRESS_GATEWAY = KUBEFLOW_ENDPOINT
NAMESPACE = "kubeflow-user-example-com"
COOKIE = auth_session["session_cookie"]
EXPERIMENT = "Default"

MINIO_ENDPOINT = "s3.amazonaws.com"
LOG_BUCKET = "emlo-s21c"

fetching auth session from https://kubeflow.ui.kubeflow.awsuser.cloudns.ph/


In [2]:
print(f"creating kfp pipeline client")

client = kfp.Client(host=INGRESS_GATEWAY + "/pipeline", cookies=COOKIE)

creating kfp pipeline client


In [3]:
import os
import requests

headers = {
    'Host': "torchserve.kubeflow-user-example-com.emlo.mmg",
    'Cookie': f"{COOKIE}",
    'Content-Type': 'application/x-www-form-urlencoded',
}

URL = 'https://kubeflow.ui.kubeflow.awsuser.cloudns.ph/v2/models'

response = requests.get(
    URL,
    headers=headers,
)

print(response.text)

{"models": ["cifar10"]}


In [5]:
import os
import requests

headers = {
    'Host': "torchserve.kubeflow-user-example-com.emlo.mmg",
    'Cookie': f"{COOKIE}",
    'Content-Type': 'application/x-www-form-urlencoded',
}

URL = 'https://kubeflow.ui.kubeflow.awsuser.cloudns.ph/v2/models/cifar10/infer'
with open('./input.json') as f:
    data = f.read().replace('\n', '').replace('\r', '').encode()

    response = requests.post(
        URL,
        headers=headers,
        data=data,
    )

print(response.text)

{"id": "6fe5e907-e8be-4b12-936f-44a45b2c5724", "model_name": "cifar10_test", "model_version": "1", "outputs": [{"name": "predict", "shape": [], "datatype": "BYTES", "data": [{"frog": 0.18918192386627197, "cat": 0.10825148969888687, "dog": 0.10372070968151093, "bird": 0.10282688587903976, "deer": 0.09945882856845856}]}]}
